In [ ]:
import torch
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
model_name = "google/flan-t5-base"
tuned_path = Path("")
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"

In [ ]:
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
result_path = Path("base_model_results")
result_path.mkdir(exist_ok=True)

In [ ]:
small_test_path = Path("../data/test_small.csv")
dataset = pd.read_csv(small_test_path)
dataset.info()

In [ ]:
PROMPT_TEMPLATE = """You're monitoring a driving assessment session. You're provided with real-time driving information including objects within 50 meters around the vehicle and the current vehicle state. The vehicle state has 4 elements:

1. throttle: a number between 0 and 0.7, where 0.7 indicates throttling
2. steering: a number in the range -1 to 1, where 0 indicates no steering
3. brake: a boolean, where True indicates that there is a brake
4. speed: current driving speed in km/h

Objects within 50 meters around the driver

Format
```
(distance between the driver and a target, a target: vehicle, pedestrian, stop sign, or traffic light)
```

Objects
```json
{}
```

Vehicle state
```json
{}
```

You are asked to give a comment on the current situation to support the driver by improving situation awareness and maintaining their high valence and medium arousal. The maximum length of comment is 10 words.

Following these steps to come up with a message:
 1. Think about the current location of the vehicle based on the provided information
 2. Think about the current situation of the vehicle based on the provided information
 3. Think if the current situation is safe or not. Does it possibly lead to hazardous events?
 4. Give a message to improve situation awareness and maintain their high valence and medium arousal
 
Output format
```json
{{
  "location": the current location,
  "situation": description of the current situation,
  "risk": risk level ("very high", "high", "medium", "low", "very low"),
  “message”: message
}}
```"""

In [ ]:
interrupted = False

try:
    for index, row in tqdm(dataset.iterrows(), total=len(dataset)):
        if result_path.joinpath(f"{index}.txt").exists():
            continue

        prompt = PROMPT_TEMPLATE.format(row["objects"], row["controls"])
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        outputs = original_model.generate(**inputs, max_length=1024)
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        result_path.joinpath(f"{index}.txt").write_text(result)

        if interrupted:
            print("Interrupt received, stopping...")
            break
except KeyboardInterrupt:
    print("Received interrupt, finishing current iteration...")
    interrupted = True